### Goal of this video
* If you have multiple accounts with multiple users how to leverage python threading to get historical data much faster

### Running a function in main thread blocks the code
* Can run in seperate "threads" to be non blocking
* Good for network/IO bound things
* Not real C like threading, more like "pseudo-threading" moving between tasks due to GIL (global interpreter lock) 

In [2]:
import threading
import time
def print_data(name,limit):
    global stop
    stop=False
    for i in range(limit):
        if stop:
            break
        time.sleep(1)
        print(f"{name} {i}")

In [2]:
print_data("ADI",10)

ADI 0
ADI 1
ADI 2
ADI 3
ADI 4
ADI 5
ADI 6
ADI 7
ADI 8
ADI 9


In [3]:
t100 = threading.Thread(target=print_data,args=('adi',100))
t10 =  threading.Thread(target=print_data,args=('bob',10))

In [4]:
t100.start()
t10.start()

In [ ]:
t100.join()
t10.join()

bob 0adi 0

adi 1
bob 1
bob 2
adi 2
bob 3
adi 3
bob 4adi 4

bob 5adi 5

bob 6adi 6

adi 7bob 7

adi 8
bob 8
adi 9
bob 9
adi 10
adi 11
adi 12
adi 13
adi 14
adi 15
adi 16
adi 17
adi 18
adi 19
adi 20
adi 21
adi 22
adi 23
adi 24
adi 25
adi 26
adi 27
adi 28
adi 29
adi 30
adi 31
adi 32
adi 33
adi 34
adi 35
adi 36
adi 37
adi 38
adi 39
adi 40
adi 41
adi 42
adi 43
adi 44
adi 45


* Use .start() to start thread
* Can use .join to block future execucution. 
* Deamon threads are threads that run in the background and don't stop the program from exiting.

In [3]:
import pandas as pd
import datetime

### Nasdaq 100 Symbols

import pandas as pd
nasdaq_100=pd.read_html("https://en.wikipedia.org/wiki/Nasdaq-100")[4]
nasdaq_100_symbols=nasdaq_100.Ticker.tolist()

### S&P 500 Symbols

sp500=pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]
sp500_symbols = sp500.Symbol.tolist()

### Russell 1000

russell=pd.read_html("https://en.wikipedia.org/wiki/Russell_1000_Index")[2]
russell_symbols=russell.Ticker.tolist()

### Combined all ticker symbols

symbol_list = list(set(nasdaq_100_symbols+sp500_symbols+russell_symbols))

##### Need to split this symbol_list into multiple parts accross ports (accounts)
* will also split across clients, but i don't think this improves performance due to IB limitations.


In [4]:
port_ids = [4001,4002,4003,4004]
client_ids = [0,1]
NUM_PORTS = len(port_ids)
NUM_CLIENTS = len(client_ids)
interval_size = len(symbol_list)//NUM_CLIENTS//NUM_PORTS
symbol_list_split={(port_ids[i],client_ids[j]):
                   symbol_list[(i*NUM_CLIENTS+j)*interval_size:(i*NUM_CLIENTS+j+1)*interval_size]
                   for i in range(NUM_PORTS) for j in range(NUM_CLIENTS)
                  
                  }

### Create function for Port/Client Combo to do work

In [5]:
from data_utils import get_historical_1m_data_fast

In [6]:
import os
os.mkdir('data')

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'data'

In [7]:
import asyncio
from ib_insync import *
def get_data(port_id,client_id):
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    ib = IB()
    ib.connect(port=port_id,clientId=client_id)
    try:
        for symbol in symbol_list_split[(port_id,client_id)]:
            print(f"Getting data using port {port_id} and clientId {client_id}")
            df=get_historical_1m_data_fast(ib,symbol,'20231101','20231201')
            df.to_csv(f'data/{symbol}_1_mins.csv')
    except Exception as e:
        print(e)
    ib.disconnect()


In [8]:
threads = []

for port_id in port_ids:
    for client_id in client_ids:
        thread = threading.Thread(target=get_data,args=(port_id,client_id))
        threads.append(thread)
        thread.start()
        

Getting data using port 4001 and clientId 0
Getting data using port 4001 and clientId 1
Getting data using port 4002 and clientId 0
Getting data using port 4004 and clientId 0
Getting data using port 4003 and clientId 0
Getting data using port 4002 and clientId 1


In [ ]:
for thread in threads:
    thread.join()

Getting data using port 4003 and clientId 1
Getting data using port 4004 and clientId 1
Retrieving data for  BPOP  for  2023-12-01
Retrieving data for  SPR  for  2023-12-01
Retrieving data for  HPE  for  2023-12-01
Retrieving data for  MPW  for  2023-12-01
Retrieving data for  PODD  for  2023-12-01
Retrieving data for  DLB  for  2023-12-01
Retrieving data for  CAG  for  2023-12-01
Retrieving data for  CARR  for  2023-12-01
Retrieving data for  SPR  for  2023-11-24
Retrieving data for  HPE  for  2023-11-24
Retrieving data for  BPOP  for  2023-11-24
Retrieving data for  PODD  for  2023-11-24
Retrieving data for  CAG  for  2023-11-24
Retrieving data for  MPW  for  2023-11-24
Retrieving data for  CARR  for  2023-11-24
Retrieving data for  PODD  for  2023-11-17
Retrieving data for  MPW  for  2023-11-17
Retrieving data for  CAG  for  2023-11-17
Retrieving data for  SPR  for  2023-11-17
Retrieving data for  HPE  for  2023-11-17
Retrieving data for  CARR  for  2023-11-17
Retrieving data for  P

reqHistoricalData: Timeout for Stock(conId=564701585, symbol='QDEL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='QDEL', tradingClass='NMS')
Error 162, reqId 2959: Historical Market Data Service error message:API historical data query cancelled: 2959, contract: Stock(conId=564701585, symbol='QDEL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='QDEL', tradingClass='NMS')


'NoneType' object has no attribute 'set_index'
No data for  2023-11-17
Retrieving data for  QDEL  for  2023-11-16


Error 162, reqId 332: Historical Market Data Service error message:Trading TWS session is connected from a different IP address, contract: Stock(conId=269174, symbol='FCNCA', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='FCNCA', tradingClass='NMS')
Error 162, reqId 103: Historical Market Data Service error message:Trading TWS session is connected from a different IP address, contract: Stock(conId=135330258, symbol='CNHI', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='CNHI', tradingClass='CNHI')


Getting data using port 4004 and clientId 1
'NoneType' object has no attribute 'set_index'
No data for  2023-11-03
Retrieving data for  FCNCA  for  2023-11-02
'NoneType' object has no attribute 'set_index'
No data for  2023-11-10
Retrieving data for  CNHI  for  2023-11-09


...

'NoneType' object has no attribute 'set_index'
No data for  2023-11-02
Getting data using port 4003 and clientId 0
'NoneType' object has no attribute 'set_index'
No data for  2023-11-09
Retrieving data for  CB  for  2023-12-01
Getting data using port 4003 and clientId 1
Retrieving data for  FBIN  for  2023-12-01
Retrieving data for  AIRC  for  2023-12-01


Error 162, reqId 335: Historical Market Data Service error message:Trading TWS session is connected from a different IP address, contract: Stock(conId=94078796, symbol='FBIN', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='FBIN', tradingClass='FBIN')
Error 162, reqId 106: Historical Market Data Service error message:Trading TWS session is connected from a different IP address, contract: Stock(conId=458022306, symbol='AIRC', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='AIRC', tradingClass='AIRC')


'NoneType' object has no attribute 'set_index'
No data for  2023-12-01
Retrieving data for  FBIN  for  2023-11-30
'NoneType' object has no attribute 'set_index'
No data for  2023-12-01
Retrieving data for  AIRC  for  2023-11-30


Error 162, reqId 336: Historical Market Data Service error message:Trading TWS session is connected from a different IP address, contract: Stock(conId=94078796, symbol='FBIN', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='FBIN', tradingClass='FBIN')
Error 162, reqId 107: Historical Market Data Service error message:Trading TWS session is connected from a different IP address, contract: Stock(conId=458022306, symbol='AIRC', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='AIRC', tradingClass='AIRC')


'NoneType' object has no attribute 'set_index'
No data for  2023-11-30
No objects to concatenate
'NoneType' object has no attribute 'set_index'
No data for  2023-11-30
No objects to concatenate
Getting data using port 4002 and clientId 0
Getting data using port 4002 and clientId 1
Retrieving data for  FTNT  for  2023-12-01
Retrieving data for  YUM  for  2023-12-01
Retrieving data for  QDEL  for  2023-11-09
Retrieving data for  GRMN  for  2023-11-03
Retrieving data for  CB  for  2023-11-24
Retrieving data for  BBWI  for  2023-11-03
Getting data using port 4001 and clientId 1
Retrieving data for  TECH  for  2023-12-01
Retrieving data for  YUM  for  2023-11-24
Retrieving data for  FTNT  for  2023-11-24
Retrieving data for  CB  for  2023-11-17
Retrieving data for  QDEL  for  2023-11-02
Retrieving data for  YUM  for  2023-11-17
Getting data using port 4001 and clientId 0
Retrieving data for  WAL  for  2023-12-01
Retrieving data for  FTNT  for  2023-11-17
Retrieving data for  TECH  for  2023